# Preperation

In [1]:
import torch
import matplotlib.pyplot as plt
import yaml
import os
import sys
from copy import deepcopy
import h5py
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import numpy as np


from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve

In [2]:
cd ..

/remote/gpu06/ernst/Master_Thesis/add_VAE/CaloINN


In [3]:
cd src

/remote/gpu06/ernst/Master_Thesis/add_VAE/CaloINN/src


In [4]:
from model import CINN
from trainer import INNTrainer, DNNTrainer, VAETrainer
import data_util
from documenter import Documenter
from plotter import Plotter
from plotting import plot_hist
from matplotlib import cm
from myDataLoader import MyDataLoader
from calc_obs import *
from model import CINN, DNN
import plotting

In [5]:
cd ..

/remote/gpu06/ernst/Master_Thesis/add_VAE/CaloINN


In [6]:
def load_trainer(directory, use_cuda=True):
    use_cuda = torch.cuda.is_available() and use_cuda
    device = 'cuda:0' if use_cuda else 'cpu' 


    with open(directory + "/params.yaml") as f:
        params = yaml.load(f, Loader=yaml.FullLoader)
        
    doc = Documenter(params['run_name'], existing_run=True, basedir=directory,
                    log_name="log_jupyter.txt", read_only=True)
    
    particle = params.get("particle_type", "piplus")

    data_path_train = os.path.join("..","..", "Datasets", particle , "train_" + particle + ".hdf5")
    params["data_path_train"] = data_path_train

    data_path_test = os.path.join("..","..", "Datasets", particle , "test_" + particle + ".hdf5")
    params["data_path_test"] = data_path_test

    classification_set = os.path.join("..","..", "Datasets", particle , "cls_" + particle + ".hdf5")
    params["classification_set"] = classification_set
    
    trainer = VAETrainer(params, device, doc)
    
    return trainer, params, device, doc

# Start calculations

In [7]:
directory = "/remote/gpu06/ernst/Master_Thesis/add_VAE/CaloINN/results/2023_01_27_test_runs/Default_setup"
trainer, params, device, doc = load_trainer(directory)
trainer.load()


Using the directory: /remote/gpu06/ernst/Master_Thesis/add_VAE/CaloINN/results/2023_01_27_test_runs/Default_setup
cuda:0


/remote/gpu06/ernst/envs/CaloINN/lib/python3.10/site-packages/torch/cuda/__init__.py:123: UserWarning: 
    Found GPU1 NVIDIA GeForce GTX TITAN which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))
/remote/gpu06/ernst/envs/CaloINN/lib/python3.10/site-packages/torch/cuda/__init__.py:123: UserWarning: 
    Found GPU2 NVIDIA GeForce GTX TITAN which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))


VAE(
  (logit_trafo_in): LogitTransformation()
  (encoder): Sequential(
    (fc0): Linear(in_features=93, out_features=80, bias=True)
    (relu0): ReLU()
    (fc1): Linear(in_features=80, out_features=80, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=80, out_features=70, bias=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=70, out_features=70, bias=True)
    (relu3): ReLU()
    (fc4): Linear(in_features=70, out_features=60, bias=True)
    (relu4): ReLU()
    (fc5): Linear(in_features=60, out_features=50, bias=True)
    (relu5): ReLU()
    (fc_mu_logvar): Linear(in_features=50, out_features=40, bias=True)
  )
  (decoder): Sequential(
    (fc0): Linear(in_features=20, out_features=50, bias=True)
    (relu0): ReLU()
    (fc1): Linear(in_features=50, out_features=60, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=60, out_features=70, bias=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=70, out_features=70, bias=True)
    (relu3): ReLU()
    (fc4